<a href="https://colab.research.google.com/github/NewAnik/DataScience/blob/master/dataframe_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 69kB/s 
     |████████████████████████████████| 204kB 40.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=0550047a7751003a05990a6076c98361ff3c6b1bf8844341645d1a94ea88bb76
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('sol').getOrCreate()

In [ ]:
df = spark.read.csv('walmart_stock.csv', header=True, inferSchema=True)

In [ ]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [ ]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [ ]:
df.head(5)

[Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

In [ ]:
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

In [ ]:
from pyspark.sql.functions import format_number

In [ ]:
result = df.describe()

In [ ]:
result.select (result['summary'], format_number(result['Open'].cast('float'),2).alias('open')).show()

+-------+--------+
|summary|    open|
+-------+--------+
|  count|1,258.00|
|   mean|   72.36|
| stddev|    6.77|
|    min|   56.39|
|    max|   90.80|
+-------+--------+



In [ ]:
df2 = df.withColumn("HV Ratio", df['High']/df['low'])
df2.select('HV Ratio').show()

+------------------+
|          HV Ratio|
+------------------+
|1.0198764326019114|
|1.0147973261342302|
|1.0214151108688558|
|1.0098522508892858|
| 1.010692481693567|
|1.0123770600203459|
|1.0082994239786682|
|1.0101009760908761|
|  1.01016781935834|
|1.0099126512096774|
|1.0063704440445784|
|1.0164016736401673|
| 1.009559947570791|
|1.0077673444973507|
|1.0205761316872428|
| 1.009338138772311|
|1.0176073720585816|
|1.0095804094882654|
| 1.016072941709128|
| 1.016341993517927|
+------------------+
only showing top 20 rows



In [ ]:
df.orderBy(df['high'].desc()).head(1)[0][0]

'2015-01-13'

In [ ]:
from pyspark.sql.functions import mean
df.select(mean("Close").alias("Average of Close")).show()

+-----------------+
| Average of Close|
+-----------------+
|72.38844998012726|
+-----------------+



In [ ]:
from pyspark.sql.functions import max, min
df.select(max("Volume").alias("max of Volume"), min("Volume").alias("min of Volume")).show()

+-------------+-------------+
|max of Volume|min of Volume|
+-------------+-------------+
|     80898100|      2094900|
+-------------+-------------+



In [ ]:
df.filter('close < 60').count()

81

In [ ]:
df.filter(df['close'] < 60).count()

81

In [ ]:
from pyspark.sql.functions import count
result = df.filter(df['close'] < 60)
result.select(count('close')).show()

+------------+
|count(close)|
+------------+
|          81|
+------------+



In [ ]:
df.filter(df['high']>80).count()/ df.count()  * 100

9.141494435612083

In [ ]:
from pyspark.sql.functions import corr
df.select(corr('high', 'volume')).show()

+-------------------+
| corr(high, volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



In [ ]:
from pyspark.sql.functions import year
yeardf = df.withColumn("Year", year(df['date']))

In [ ]:
max_df = yeardf.groupby('Year').max()

In [ ]:
max_df.select('Year', 'max(high)').sort("Year").show()

+----+---------+
|Year|max(high)|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



In [ ]:
from pyspark.sql.functions import month
monthdf = df.withColumn('Month', month('date'))
monthavg = monthdf.select(['month', 'close']).groupby('month').mean()
monthavg.select('Month', 'avg(close)').orderBy('Month').show()

+-----+-----------------+
|Month|       avg(close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

